# Energy calibration
このノートでは、検出器のraw ADCの値をch毎に線形関数でキャリブレーションし、エネルギー情報に変換する。
キャリブレーションパラメータはCSVファイルで準備済みとする。

- data/clover_data.parquet:
こちらは、クローバー型ゲルマニウム検出器3台からなるデータで、各検出器に4つの読み出しチャンネルがあり、
合計12chとなっている。

- ClvCalib.csv: キャリブレーションパラメータ

## ファイルを開く

In [20]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.getOrCreate()
df = spark.read.parquet("./data/clover_data.parquet")
df_select = df.select("ts","cfd","ch","adc")
df_select.show(5)

+-------------+-----+---+----+
|           ts|  cfd| ch| adc|
+-------------+-----+---+----+
|1657567439657|23504|  8|1348|
|1657568832534| 5435|  6|6750|
|1657569292316|12850|  2| 744|
|1657569301206|14372|  7| 432|
|1657569615821|11872| 11| 900|
+-------------+-----+---+----+
only showing top 5 rows


とりあえず"ch"と"adc"列だけセレクト
## 整数値を実数値に変換する。
raw ADCの値は整数値なので、0から1の乱数を足して実数値にする。

In [21]:
df_real = df_select.withColumn("adc_real", F.col("adc")+F.rand())
df_real.show(5)

+-------------+-----+---+----+------------------+
|           ts|  cfd| ch| adc|          adc_real|
+-------------+-----+---+----+------------------+
|1657567439657|23504|  8|1348| 1348.639293189631|
|1657568832534| 5435|  6|6750| 6750.340310900506|
|1657569292316|12850|  2| 744|  744.286927331077|
|1657569301206|14372|  7| 432|432.28702890550653|
|1657569615821|11872| 11| 900| 900.5425011924168|
+-------------+-----+---+----+------------------+
only showing top 5 rows


## CSVからキャリブレーションパラメータのデータフレームを作る
CSVファイルがあるとする。
```csv
ch,p0,p1
0,0.2149837759,0.0803461538
1,0.1280723048,0.2546962
...
```
spark.read.csv() でCSVからデータフレームを作成。
`header=True`は一行目を列の名前として読むオプション。
`inferSchema=True`はデータをstringではなく、推論した型で読み込むオプション。

In [22]:
param_df = spark.read.csv("./ClvCalib.csv", header=True, inferSchema=True)
param_df.show()

+---+--------------------+-------------------+
| ch|                  p0|                 p1|
+---+--------------------+-------------------+
|  0|        0.2149837759|       0.0803461538|
|  1|        0.1280723048|          0.2546962|
|  2|       -0.1933825665|       0.2941007359|
|  3|        0.0611566264|       0.0929934353|
|  4|       -0.0563791084|       0.2946970847|
|  5|        0.0371477634|       0.1212543693|
|  6| -0.3015558027966563|0.29948038289716816|
|  7|-0.09316673031071332|0.28713228807294117|
|  8|   65.26576358770785|0.13040375588633496|
|  9| 0.07402471776299535|0.34475015560367683|
| 10| 0.03482990048200918| 0.3121818201909335|
| 11|0.005223987900080829|0.30955157952139184|
+---+--------------------+-------------------+



## ch番号が一致するパラメータを元のデータフレームにくっつける。
- join(): 二つのデータフレームを`on=`で指定した条件を満たす行同士で結合する。`how="inner"`は両方のデータフレームの行が存在するものだけ出力、`how="outer"`は片方が存在しない行も残す。

In [23]:
calib_df = df_real.join(param_df, on=["ch"], how="inner")
calib_df.show(5)

+---+-------------+-----+----+------------------+--------------------+-------------------+
| ch|           ts|  cfd| adc|          adc_real|                  p0|                 p1|
+---+-------------+-----+----+------------------+--------------------+-------------------+
|  8|1657567439657|23504|1348| 1348.639293189631|   65.26576358770785|0.13040375588633496|
|  6|1657568832534| 5435|6750| 6750.340310900506| -0.3015558027966563|0.29948038289716816|
|  2|1657569292316|12850| 744|  744.286927331077|       -0.1933825665|       0.2941007359|
|  7|1657569301206|14372| 432|432.28702890550653|-0.09316673031071332|0.28713228807294117|
| 11|1657569615821|11872| 900| 900.5425011924168|0.005223987900080829|0.30955157952139184|
+---+-------------+-----+----+------------------+--------------------+-------------------+
only showing top 5 rows


p0、p1の列に、ch番号に対応する値が入った。
## パラメータを使ってエネルギーに変換
ここまでできればあとは新しくadc_calを定義するだけ。

In [24]:
calib_df = calib_df.withColumn("adc_cal", F.expr("p0 + adc_real * p1"))
calib_df.show(5)

+---+-------------+-----+----+------------------+--------------------+-------------------+------------------+
| ch|           ts|  cfd| adc|          adc_real|                  p0|                 p1|           adc_cal|
+---+-------------+-----+----+------------------+--------------------+-------------------+------------------+
|  8|1657567439657|23504|1348| 1348.639293189631|   65.26576358770785|0.13040375588633496| 241.1333927555278|
|  6|1657568832534| 5435|6750| 6750.340310900506| -0.3015558027966563|0.29948038289716816| 2021.292945191876|
|  2|1657569292316|12850| 744|  744.286927331077|       -0.1933825665|       0.2941007359|218.70195048231957|
|  7|1657569301206|14372| 432|432.28702890550653|-0.09316673031071332|0.28713228807294117|124.03039698358103|
| 11|1657569615821|11872| 900| 900.5425011924168|0.005223987900080829|0.30955157952139184| 278.7695776581576|
+---+-------------+-----+----+------------------+--------------------+-------------------+------------------+
only showi

## ファイルに書き出す
- write.parquet(): parquetファイルにデータフレームを書き出す。
- mode("overwrite"): ファイルが存在する場合、上書きする。

In [26]:
calib_df.select("ts","ch","cfd","adc_cal").write.mode("overwrite").parquet("./data/clover_caldata.parquet")